In [1]:
import csv
import re
from collections import OrderedDict

In [56]:
# counts number of rows
with open('yourFile.csv', "r", encoding="utf-8") as file:
    readCSV = csv.reader(file)
    lines = len(list(readCSV))
    
file.close()

In [57]:
def saveSameBankName(equalBanks, rowBefore, row):
    if not equalBanks:
            equalBanks += [rowBefore]
    equalBanks += [row]
    return equalBanks

def saveSameBICs(equalBanks, presentBIC):
    # check for emtpy BIC (has to be at least one BIC!)
    for line in equalBanks: 
        if line[2] != "":             
            presentBIC += [line]
    
    return presentBIC

def leaveOutRedundantLine(equalBanks, presentBIC):
    differentBankCodeBuffer = []
    
    for line in equalBanks:
        countingPrintinginFile = 0
        differentBankCode = 0
        counterDifferentBankCode = 1
        counterSameBankCode = 0

        if presentBIC: 
        # there is at least one BIC
            for bicLine in presentBIC:
                if countingPrintinginFile == 1:
                    print()
                elif line[5] == bicLine[5] and line[2] != "":   # bankcode same, BIC not empty
                    countingPrintinginFile = 1                  # also: different bankcode, BIC not empty
                    writer.writerow(line)
                elif line[5] == bicLine[5] and line[2] == "":   # bankcode same, BIC empty
                    countingPrintinginFile = 1
                elif line[5] != bicLine[5] and line[2] == "":   # bankcode different, BIC empty
                    differentBankCode = differentBankCode + 1

                if differentBankCode == len(presentBIC):
                    differentBankCodeBuffer += [line]
        else:
            # There are no BICs at all
            if len(equalBanks) == 1:
                writer.writerow(line)
            else:
                for lineAfter in equalBanks:
                    if line[5] != lineAfter[5]:
                        differentBankCodeBuffer += [line]
                    else:
                        counterSameBankCode = counterSameBankCode + 1
                    
                    if counterSameBankCode == len(equalBanks):
                        differentBankCodeBuffer += [line]
        

    # doesn't print duplicates from "bankcode different"
    if differentBankCodeBuffer != "":
        differentBankCodeBuffer = OrderedDict((tuple(x), x) for x in differentBankCodeBuffer).values()
        
        for items in differentBankCodeBuffer:
            writer.writerow(items)

        differentBankCodeBuffer = []

                   
with open('yourFile.csv', "r", encoding="utf-8") as file:
    readCSV = csv.reader(file)
    
    # output will be saved in new file
    noRedundancy = open('resultingFile_noRedundancy.csv', 'w')
    writer = csv.writer(noRedundancy)
    bankName = ""; bic = ""; bankCode = ""; rowBefore = "" 
    equalBanks = []
    counter = 0
    presentBIC = []
    counterForComma = 0
    counterForLetter = 1
    newRowBuffer = ""
    
    for row in readCSV:
        counter = counter + 1
        
        ## the file may have an error -> if there are eight commas in one row it means that the
        ## bankName has a comma inside (but CSV separates it) so the BIC has a part of the BankName
        ## TODO
        
                    
        if not equalBanks:
            if rowBefore:
                equalBanks = [rowBefore]
        
        # same bank name
        if bankName == row[1]:
            equalBanks = saveSameBankName(equalBanks, rowBefore, row)
            
        elif bankName in row[1] and ("(Gf P2)" in row[1] or "(Gf P3)" in row[1]):
            equalBanks = saveSameBankName(equalBanks, rowBefore, row)
        
        else:
            if "(Gf P2)" in bankName:
                bankName = re.sub("Gf P2", "", bankName)
                bankName = re.sub("[()]", "", bankName)
                
            elif "(Gf P3)" in bankName:
                bankName = re.sub("Gf P3", "", bankName)
                bankName = re.sub("[()]", "", bankName)
            
            print("in else vor if")
            # looks in array of same banks for redundancy
            if str(bankName) in str(equalBanks):
                presentBIC = saveSameBICs(equalBanks, presentBIC)
                leaveOutRedundantLine(equalBanks, presentBIC)

            # different bank names
            else: 
                writer.writerow(rowBefore)

            equalBanks = []    
            presentBIC = []
               
        bankName = row[1]
        bic = row[2]
        bankCode = row[5]
        rowBefore = row
        print()
        
        
         # handle last row
        if counter == lines:
            if re.search(bankName, str(equalBanks)):    
                presentBIC = saveSameBICs(equalBanks, presentBIC)
                leaveOutRedundantLine(equalBanks, presentBIC)

            # different bank names
            else: 
                writer.writerow(rowBefore)

            equalBanks = [] 
            presentBIC = []